In [30]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
import re

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.9.3
urllib -> 3.8
pandas -> 1.2.4


In [32]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
        
        #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  
        
        
        #acabamento
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
         
        cards.append(card)


        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dados/dataset-casas-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,acabamento
0,1595000,LEARDI 100 anos de tradição imobiliária tem ...,Rua Afonso Celso,Vila Mariana,3,2,3,140,442,0,Super destaque,1,1
1,350000,02 CASAS VILA CISPER CASA 01: 01 dormitório...,Rua Governador Archer,Vila Cisper,1,1,2,95,0,0,Super destaque,0,0
2,2200000,Um terreno com 980 m² com escrituras separadas...,Lagoa da Conceição,Florianópolis,5,12,5,270,140,0,Super destaque,0,0
3,400000,Grande oportunidade Entrada facilitada Utili...,Rua das Boas Noites,Centro,3,3,2,140,0,0,Nenhuma,1,0
4,310000,Grande oportunidade Entrada facilitada Utilize...,Rua Adamantina,Vila Dona Paulina,3,2,2,120,0,0,Nenhuma,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,560000,"Casa com 3 dormitorios, 178 m/ 269 m terreno a...",Rua Dario Pompeu de Camargo,Parque São Quirino,3,4,3,178,194,0,Nenhuma,1,0
9596,250000,"CASA COM 02 QUARTOS, SALA, COZINHA, 01 BANHEIR...",Residencial Oliveira,Alfenas,2,2,1,10,0,0,Nenhuma,0,0
9597,295000,"Casa térrea, nova pronta para morar , no bairr...",Parque Residencial Sabiás,Indaiatuba,2,1,1,52,0,0,Nenhuma,0,0
9598,650000,Casa na Coroa do Meio andar térreo: 3 quartos ...,Rua Jornalista João Batista de Sant'Anna,Coroa do Meio,4,2,2,480,0,0,Nenhuma,1,0


In [34]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(350)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
         #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)    
                        
        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dados/dataset-ap-geral.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
academia         0
varanda          0
transporte       0
salao            0
planejado        0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,academia,varanda,transporte,salao,planejado
0,753000,Em Construção Entrega em Setembro de 2022. O...,Vila Guilhermina,São Paulo,3,1,2,84,150,550,Super destaque,0,0,0,0,0,0
1,2590000,Uma JOIA te esperando ! PORTEIRA FECHADA ! O ...,Boqueirão,Santos,3,3,4,167,520,1200,Super destaque,0,0,0,0,0,0
2,579000,Venha morar 08 minutos a pé do metrô Santos Im...,Ipiranga,São Paulo,2,1,1,54,0,540,Super destaque,1,0,0,1,0,0
3,350000,* OPORTUNIDADE UNICA NA VILA LAURA * 3 QUART...,Vila Laura,Salvador,3,1,2,80,450,600,Destaque,1,0,1,0,0,0
4,869000,O apartamento está localizado no bairro Jardim...,Rua Corifeu de Azevedo Marques,Jardim das Indústrias,3,2,3,122,0,0,Nenhuma,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,190000,"Apartamento com 02 quartos, sala, cozinha, ban...",Califórnia,Belo Horizonte,2,1,1,50,1,220,Nenhuma,0,0,0,0,0,0
8396,240000,APARTAMENTO NA PENHA DE FRANÇA 02 Dormitóri...,Avenida Gabriela Mistral,Penha,2,1,1,44,0,550,Destaque,0,0,0,0,0,1
8397,170000,Apartamento em Jardim Coleginho/SP de 50m². 2 ...,Condomínio Residencial São Miguel,Jardim Coleginho,2,1,1,50,0,240,Nenhuma,0,0,0,0,0,0
8398,370000,"02 Dormitórios, sendo 01 suíte, Banheiro socia...",Rua Elias Rodrigues Claro,Jardim Sao Carlos,2,1,2,55,0,0,Destaque,1,0,1,0,0,1


In [37]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(300)


##interando por todas as paginas do site
for i in range(pages):
    
    ##Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/rj+rio-de-janeiro/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='simple-card__box')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quarto-min'], card['quarto-max'] = '1','1' 
        elif '-' in quartos.get_text() :
            card['quarto-min'], card['quarto-max'], *outro = quartos.get_text().replace(' ','').replace('\n','').split('-')
        else:        
           card['quarto-min'], card['quarto-max'] = quartos.get_text().replace(' ','').replace('\n',''),quartos.get_text().replace(' ','').replace('\n','')    
        
        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem-min'], card['garagem-max'] = '0' , '0'
        elif '-' in garagem.get_text() :
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','') , garagem.get_text().replace(' ','').replace('\n','')

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro-min'], card['banheiro-max'] = '1','1'
        elif '-' in banheiro.get_text() :
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n',''),banheiro.get_text().replace(' ','').replace('\n','')

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)-min'], card['area(m2)-max'] = '10','30'
        elif '-' in metro.get_text() :
            card['area(m2)-min'], card['area(m2)-max'], *outro = metro.get_text().replace(' ','').replace('\n','').replace('m²','').split('-')
        else:
            card['area(m2)-min'], card['area(m2)-max'] = metro.get_text().replace(' ','').replace('\n','').replace('m²',''),metro.get_text().replace(' ','').replace('\n','').replace('m²','')

            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  
        
        
        #acabamento
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
         
        cards.append(card)

        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dados/dataset-casas-rj.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

valor            0
descricao        0
local            0
bairro/cidade    0
quarto-min       0
quarto-max       0
garagem-min      0
garagem-max      0
banheiro-min     0
banheiro-max     0
area(m2)-min     0
area(m2)-max     0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quarto-min,quarto-max,garagem-min,garagem-max,banheiro-min,banheiro-max,area(m2)-min,area(m2)-max,iptu,condominio,novidade,suite,acabamento
0,6850000,Casa única no Alto Jardim Botânico! Uma casa ...,Rua Peri,Jardim Botânico,4,4,2,2,8,8,508,610,0,0,Nenhuma,1,0
1,1999000,O Condomínio Residencial Camboata está a pouco...,Rua Geógrafo Amora,Piratininga,3,5,3,4,4,6,400,700,0,0,Nenhuma,1,0
2,2990000,Mansões Rio Mar. Excelentes projetos de casas ...,Avenida das Américas,Barra da Tijuca,5,5,3,3,6,6,460,540,0,0,Nenhuma,1,0
3,1195000,BARRA 2 IMÓVEIS VENDE CASA EM JACAREPAGUÁ ...,Rua Rio dos Passarinhos,Anil,4,4,3,3,6,6,311,311,367,680,Nenhuma,1,0
4,330000,Maravilhosa casa Linear em condomínio fechado ...,Rua Campina Grande,Campo Grande,3,3,2,2,1,1,100,100,0,300,Nenhuma,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,530000,Casa com 3 dormitórios à venda por R$ 530.000 ...,Rua Azulão,Marechal Hermes,3,3,2,2,2,2,84,84,0,0,Nenhuma,0,0
7196,500000,Cód: 559982. Aceitamos financiamento e FGTS. D...,Vila da Penha,Rio de Janeiro,3,3,2,2,4,4,221,221,0,0,Nenhuma,0,0
7197,6750000,"Empreendimento de alto padrão, para a construç...",Avenida Jorge Curi,Barra da Tijuca,5,5,4,4,6,6,656,656,0,1800,Nenhuma,0,0
7198,1008000,"Linda Casa Triplex (160m²), com 3 quartos, suí...",Rua Engenheiro Gama Lobo,Vila Isabel,3,3,2,2,4,4,160,160,1532,772,Nenhuma,1,0


In [40]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(300)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = "https://www.zapimoveis.com.br/venda/?pagina="+str(i)+"&tipo=Im%C3%B3vel%20usado&transacao=Venda"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

        #tipo    
        if re.search("kitnet|kitnets|kit|kit net|kitinete|kitinetes|quitinete|kitão|germinada|geminada|conjugado", card['descricao'].lower()):    
            card['tipo'] =  int(1)
        elif re.search("apartamento|apartamentos|apto|cobertura", card['descricao'].lower()):
            card['tipo'] =  int(2)
        elif re.search("casa de condominio|casa de condomínio|casa de condominios|casa de condomínios", card['descricao'].lower()):
            card['tipo'] =  int(3)    
        elif re.search("studio|studios", card['descricao'].lower()):
            card['tipo'] =  int(4)
        elif re.search("vila|villa", card['descricao'].lower()):    
            card['tipo'] =  int(5)
        elif re.search("casa|casas", card['descricao'].lower()):
            card['tipo'] =  int(6)    
        elif re.search("loft|duplex|dupllex", card['descricao'].lower()):    
            card['tipo'] =  int(7)
        elif re.search("loteamento|lote|terrenos|terreno|trrno|lote/terreno", card['descricao'].lower()):    
            card['tipo'] =  int(8)
        else:
            card['tipo'] =  int(9)
                
       #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)  
            
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
                        
        cards.append(card)

        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dados/dataset-imovel-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
tipo             0
suite            0
academia         0
varanda          0
transporte       0
salao            0
planejado        0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,tipo,suite,academia,varanda,transporte,salao,planejado,acabamento
0,950000,"Apartamento amplo, 190m2, 3 ambientes principa...",Rua Itamira,Vila Andrade,4,4,4,191,929,2941,Super destaque,2,1,0,1,0,0,0,0
1,4500000,Lindas casas para venda na mais charmosa Praia...,Avenida Mãe Bernarda,Praia de Juquehy,4,2,5,310,0,0,Super destaque,6,0,0,0,0,0,0,0
2,965000,CASA A VENDA NO CONDOMÍNIO CHÁCARA DAS PALMEIR...,Medeiros,Jundiaí,3,3,3,140,1152,535,Super destaque,6,1,0,0,0,0,0,0
3,350000,Apartamento Parque Industrial Residencial ...,Rua Teresina,Parque Industrial,2,1,2,64,180,350,Nenhuma,2,1,0,0,0,0,0,0
4,240000,QUARTOS 2 sedo 1 suite GARAGEM 1 DESCRIÇÃO O ...,Rua Desembargador Cid Campelo,Cidade Industrial,2,1,2,64,20,299,Nenhuma,9,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,465000,Apartamento no Condomínio Floradas de São José...,Jardim Satélite,São José dos Campos,2,0,2,77,0,0,Nenhuma,2,1,0,0,1,0,1,1
7196,171000,Casa localizada no bairro FORQUILHAS | SÃO JOS...,Rua Jair Coan,Forquilhas,2,1,1,55,0,30,Nenhuma,6,0,0,0,0,0,0,0
7197,220000,"NBKT NOBILITARE oferta para VENDA, Apartamen...",Centro Histórico,Porto Alegre,1,0,1,45,0,272,Nenhuma,2,0,0,0,0,0,0,0
7198,260000,"NBKT NOBILITARE oferta para VENDA, Apartamen...",Centro Histórico,Porto Alegre,1,0,1,55,0,350,Destaque,2,0,0,0,0,0,0,0
